In [3]:
# test.py
import os
import tensorflow as tf
import numpy as np
import cv2

# module-level variables ##############################################################################################
RETRAINED_LABELS_TXT_FILE_LOC = os.getcwd() + "/" + "retrained_labels.txt"
RETRAINED_GRAPH_PB_FILE_LOC = os.getcwd() + "/" + "retrained_graph.pb"

TEST_IMAGES_DIR = os.getcwd() + "/specs/_test"

SCALAR_RED = (0.0, 0.0, 255.0)
SCALAR_BLUE = (255.0, 0.0, 0.0)

TP = {
    'classical': 0,
    'hiphop': 0,
    'jazz': 0,
    'metal': 0,
    'pop': 0,
    'reggae': 0
}
FP = {
    'classical': 0,
    'hiphop': 0,
    'jazz': 0,
    'metal': 0,
    'pop': 0,
    'reggae': 0
}
FN = {
    'classical': 0,
    'hiphop': 0,
    'jazz': 0,
    'metal': 0,
    'pop': 0,
    'reggae': 0
}

overallConfidence = {
    'classical': 0,
    'hiphop': 0,
    'jazz': 0,
    'metal': 0,
    'pop': 0,
    'reggae': 0
}

confidenceWhenRight = {
    'classical': 0,
    'hiphop': 0,
    'jazz': 0,
    'metal': 0,
    'pop': 0,
    'reggae': 0
}

confidenceWhenWrong = {
    'classical': 0,
    'hiphop': 0,
    'jazz': 0,
    'metal': 0,
    'pop': 0,
    'reggae': 0
}

dictionary = {}

#######################################################################################################################
def main():
    print("starting program . . .")

    if not checkIfNecessaryPathsAndFilesExist():
        return
    # end if
#######################################################################################################################


#######################################################################################################################

    # get a list of classifications from the labels file
    classifications = []
    # for each line in the label file . . .
    for currentLine in tf.gfile.GFile(RETRAINED_LABELS_TXT_FILE_LOC):
        # remove the carriage return
        classification = currentLine.rstrip()
        # and append to the list
        classifications.append(classification)
    # end for

    # show the classifications to prove out that we were able to read the label file successfully
    print("classifications = " + str(classifications))

    # load the graph from file
    with tf.gfile.FastGFile(RETRAINED_GRAPH_PB_FILE_LOC, 'rb') as retrainedGraphFile:
        # instantiate a GraphDef object
        graphDef = tf.GraphDef()
        # read in retrained graph into the GraphDef object
        graphDef.ParseFromString(retrainedGraphFile.read())
        # import the graph into the current default Graph, note that we don't need to be concerned with the return value
        _ = tf.import_graph_def(graphDef, name='')
    # end with

    # if the test image directory listed above is not valid, show an error message and bail
    if not os.path.isdir(TEST_IMAGES_DIR):
        print("the test image directory does not seem to be a valid directory, check file / directory paths")
        return
    # end if

    with tf.Session() as sess:
        # for each file in the test images directory . . .
        for fileName in os.listdir(TEST_IMAGES_DIR):
            # if the file does not end in .jpg or .jpeg (case-insensitive), continue with the next iteration of the for loop
            if not (fileName.lower().endswith(".jpg") or fileName.lower().endswith(".jpeg")):
                continue
            # end if

            # show the file name on std out
            print("\n" + fileName)

            # get the file name and full path of the current image file
            imageFileWithPath = os.path.join(TEST_IMAGES_DIR, fileName)
            # attempt to open the image with OpenCV
            openCVImage = cv2.imread(imageFileWithPath)

            # if we were not able to successfully open the image, continue with the next iteration of the for loop
            if openCVImage is None:
                print("unable to open " + fileName + " as an OpenCV image")
                continue
            # end if

            # get the final tensor from the graph
            finalTensor = sess.graph.get_tensor_by_name('final_result:0')

            # convert the OpenCV image (numpy array) to a TensorFlow image
            tfImage = np.array(openCVImage)[:, :, 0:3]
            
            # run the network to get the predictions
            predictions = sess.run(finalTensor, {'DecodeJpeg:0': tfImage})

            # sort predictions from most confidence to least confidence
            sortedPredictions = predictions[0].argsort()[-len(predictions[0]):][::-1]

            print("---------------------------------------")

            # keep track of if we're going through the next for loop for the first time so we can show more info about
            # the first prediction, which is the most likely prediction (they were sorted descending above)
            onMostLikelyPrediction = True
            # for each prediction . . .
            for prediction in sortedPredictions:
                strClassification = classifications[prediction]

                # if the classification (obtained from the directory name) ends with the letter "s", remove the "s" to change from plural to singular
                if strClassification.endswith("s"):
                    strClassification = strClassification[:-1]
                # end if

                # get confidence, then get confidence rounded to 2 places after the decimal
                confidence = predictions[0][prediction]

                # if we're on the first (most likely) prediction, state what the object appears to be and show a % confidence to two decimal places
                if onMostLikelyPrediction:
                    # get the score as a %
                    scoreAsAPercent = confidence * 100.0
                    # show the result to std out
                    print("the object appears to be " + strClassification + ", " + "{0:.2f}".format(scoreAsAPercent) + "% confidence")
                    # write the result on the image
#                     writeResultOnImage(openCVImage, strClassification + ", " + "{0:.2f}".format(scoreAsAPercent) + "% confidence")
                    # finally we can show the OpenCV image
#                     cv2.imshow(fileName, openCVImage)
                    # mark that we've show the most likely prediction at this point so the additional information in
                    # this if statement does not show again for this image
                    dictionary["{}".format(strClassification)] = "{0:.2f}".format(scoreAsAPercent)
                    overallConfidence[strClassification] += scoreAsAPercent
            
                    for genre in classifications:
                        if genre in fileName:
                            GENRE = genre
                    if strClassification in fileName:
                        confidenceWhenRight[strClassification] += scoreAsAPercent
                        TP[strClassification] += 1
                    else:
                        confidenceWhenWrong[GENRE] += scoreAsAPercent
                        FP[strClassification] += 1
                        FN[GENRE] +=1    
            
                onMostLikelyPrediction = False
                # end if
#######################################################################################################################

    
#######################################################################################################################
                
                # for any prediction, show the confidence as a ratio to five decimal places
                print(strClassification + " (" +  "{0:.5f}".format(confidence) + ")")
            # end for

            # pause until a key is pressed so the user can see the current image (shown above) and the prediction info
            cv2.waitKey()
            # after a key is pressed, close the current window to prep for the next time around
            cv2.destroyAllWindows()
        # end for
    # end with

    # write the graph to file so we can view with TensorBoard
    tfFileWriter = tf.summary.FileWriter(os.getcwd())
    tfFileWriter.add_graph(sess.graph)
    tfFileWriter.close()
#     print("\n Final Results")
#     print("-------------------------------------------------")
#     print(TP)
#     print(FP)
#     print(FN)
# end main

#######################################################################################################################
def checkIfNecessaryPathsAndFilesExist():
    if not os.path.exists(TEST_IMAGES_DIR):
        print('')
        print('ERROR: TEST_IMAGES_DIR "' + TEST_IMAGES_DIR + '" does not seem to exist')
        print('Did you set up the test images?')
        print('')
        return False
    # end if

    if not os.path.exists(RETRAINED_LABELS_TXT_FILE_LOC):
        print('ERROR: RETRAINED_LABELS_TXT_FILE_LOC "' + RETRAINED_LABELS_TXT_FILE_LOC + '" does not seem to exist')
        return False
    # end if

    if not os.path.exists(RETRAINED_GRAPH_PB_FILE_LOC):
        print('ERROR: RETRAINED_GRAPH_PB_FILE_LOC "' + RETRAINED_GRAPH_PB_FILE_LOC + '" does not seem to exist')
        return False
    # end if

    return True
# end function

if __name__ == "__main__":
    main()

starting program . . .
classifications = ['classical', 'hiphop', 'jazz', 'metal', 'pop', 'reggae']

classical.00000.jpg
---------------------------------------
the object appears to be classical, 41.08% confidence
classical (0.41081)
pop (0.22580)
jazz (0.17063)
reggae (0.13048)
hiphop (0.03261)
metal (0.02966)

classical.00001.jpg
---------------------------------------
the object appears to be classical, 33.55% confidence
classical (0.33547)
pop (0.27737)
jazz (0.27174)
reggae (0.06487)
metal (0.03791)
hiphop (0.01263)

classical.00002.jpg
---------------------------------------
the object appears to be jazz, 32.67% confidence
jazz (0.32667)
pop (0.26611)
classical (0.22908)
metal (0.08451)
reggae (0.06629)
hiphop (0.02733)

classical.00003.jpg
---------------------------------------
the object appears to be classical, 70.56% confidence
classical (0.70559)
jazz (0.16270)
pop (0.10036)
reggae (0.02293)
metal (0.00589)
hiphop (0.00253)

classical.00004.jpg
-----------------------------

---------------------------------------
the object appears to be metal, 66.67% confidence
metal (0.66674)
jazz (0.12315)
pop (0.09923)
hiphop (0.07362)
reggae (0.03539)
classical (0.00187)

pop.00000.jpg
---------------------------------------
the object appears to be pop, 46.82% confidence
pop (0.46816)
jazz (0.33733)
metal (0.07517)
hiphop (0.05586)
reggae (0.05504)
classical (0.00844)

pop.00001.jpg
---------------------------------------
the object appears to be pop, 66.75% confidence
pop (0.66749)
hiphop (0.15521)
reggae (0.07862)
jazz (0.05172)
metal (0.04558)
classical (0.00139)

pop.00002.jpg
---------------------------------------
the object appears to be pop, 32.91% confidence
pop (0.32907)
jazz (0.30415)
metal (0.26450)
hiphop (0.03724)
reggae (0.03566)
classical (0.02938)

pop.00003.jpg
---------------------------------------
the object appears to be metal, 64.61% confidence
metal (0.64613)
pop (0.13190)
jazz (0.10740)
hiphop (0.06779)
reggae (0.04538)
classical (0.00141)



In [4]:
print("True positives")
print(TP)
print("False negatives")
print(FN)
print("False positives")
print(FP)

TN = {
'classical': 0,
'hiphop': 0,
'jazz': 0,
'metal': 0,
'pop': 0,
'reggae': 0
}

for genre in TN.keys():
    TN[genre] = 50 - FP[genre]
print("True Negatives")
print(TN)

accuracy = 0
total = 0

for genre in TP.keys():
    accuracy += TP[genre]
    total += TP[genre]

for genre in FP.keys():
    total += FN[genre]

accuracy /= (total/100)
# print(total)
print("Accuracy = " + str(accuracy))

# for genre in overallConfidence.keys():
#     overallConfidence[genre] /= 10
#     confidenceWhenWrong[genre] /= 10
#     confidenceWhenRight[genre] /= 10

# print(overallConfidence)
# print(confidenceWhenWrong)
# print(confidenceWhenRight)

True positives
{'classical': 9, 'hiphop': 9, 'jazz': 8, 'metal': 8, 'pop': 3, 'reggae': 6}
False negatives
{'classical': 1, 'hiphop': 1, 'jazz': 2, 'metal': 2, 'pop': 7, 'reggae': 4}
False positives
{'classical': 1, 'hiphop': 0, 'jazz': 8, 'metal': 5, 'pop': 3, 'reggae': 0}
True Negatives
{'classical': 49, 'hiphop': 50, 'jazz': 42, 'metal': 45, 'pop': 47, 'reggae': 50}
Accuracy = 71.66666666666667


In [5]:
precision = {
'classical': 0,
'hiphop': 0,
'jazz': 0,
'metal': 0,
'pop': 0,
'reggae': 0
}
recall = {
'classical': 0,
'hiphop': 0,
'jazz': 0,
'metal': 0,
'pop': 0,
'reggae': 0
}

for genre in precision.keys():
    precision[genre] = TP[genre] / (TP[genre] + FP[genre])
    precision[genre] = "{0:.2f}".format(precision[genre])
    recall[genre] = TP[genre] / (TP[genre] + FN[genre])
    recall[genre] = "{0:.2f}".format(recall[genre])
    
print("Precision")
print(precision)
print("Recall")
print(recall)

Precision
{'classical': '0.90', 'hiphop': '1.00', 'jazz': '0.50', 'metal': '0.62', 'pop': '0.50', 'reggae': '1.00'}
Recall
{'classical': '0.90', 'hiphop': '0.90', 'jazz': '0.80', 'metal': '0.80', 'pop': '0.30', 'reggae': '0.60'}
